In [1]:
# %load process_tweet
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('display.max_rows', 100)

from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def process_tweet(tweet, tknzr, advanced=False):
    st_1 = []
    for w in tknzr.tokenize(tweet):
        #remove retweet annotation if present:
        if w == 'RT':
            if advanced:
                st_1.append('rt')
        elif w[0] == '@':
            if advanced:
                st_1.append('<user>')
        #remove hashtag symbol
        elif w[0] == '#':
            st_1.append(w[1:])
        #replace link with LINK keyword
        elif w[:4] == 'http':
            st_1.append('<url>')
        elif w.isnumeric():
            if advanced:
                st_1.append('<number>')
        else:
            st_1.append(w)
    
    st_2 = []
    
    #remove stop words and punctuation, make everything lowercase
    if advanced:
        st_2 = [w.lower() for w in st_1 if is_valid_token(w) and 
                    not w.lower() in stop_words]
    else:
        st_2 = [w.lower() for w in st_1 if w.isalpha() and
                not w.lower() in stop_words]
    
    #lemmatization (converts all words to root form for standardization)
    lem = WordNetLemmatizer()
    st_3 = list(map(lambda x: lem.lemmatize(x, pos='v'), st_2))
    
    #now do word segmentation/spell check
    return ' '.join(st_3)

[nltk_data] Downloading package wordnet to /home/ashwin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
#now read in raw labeled tweets
df = pd.read_csv('data/labeled_prelim.csv').dropna()
df = df.astype({'Relevancy':np.int32, 'Urgency':np.int32})
df = df.reset_index(drop=True)
df.pop('Id')
df.head()

,Text,Relevancy,Urgency
0,More millions in #Afghanistan even with ZERO a...,0,0
1,These are the last post my brother made on soc...,2,1
2,In @cityofcc listening to local officials abou...,0,0
3,So so so damn proud of @5ugarcane who is tirel...,3,0
4,How can you help with #Harvey disaster respons...,0,0


In [7]:
#now preprocess the tweets
# sym_spell = create_symspell(2,7,'data/frequency_dictionary_en_82_765.txt')
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
df['Text'] = df['Text'].map(lambda x: process_tweet(x, tknzr, False))
df.head()

,Text,Relevancy,Urgency
0,millions afghanistan even zero attack isis sym...,0,0
1,last post brother make social media phone go v...,2,1
2,listen local officials epa help harvey response,0,0
3,damn proud tirelessly help fellow texans affec...,3,0
4,help harvey disaster response help victims nat...,0,0


Our next goal is to extract a bunch of features from the data and use feature selection methods to reduce the dimensionality of the data. This work is based on the research of Krouska et al. These are the features that will be extracted:
* tf-idf
* n-gram

In [10]:
#calculate tf-idf features and add to dataframe
vectorizer = TfidfVectorizer()
vecs = vectorizer.fit_transform(df['Text']).toarray()
labels = list(map(lambda x: 'tf-idf: ' + str(x), range(vecs.shape[1])))

df2 = pd.DataFrame(vecs, columns=labels)
df2['Relevancy'] = df['Relevancy']
df2['Urgency'] = df['Urgency']
df2.head()

,tf-idf: 0,tf-idf: 1,tf-idf: 2,tf-idf: 3,tf-idf: 4,tf-idf: 5,tf-idf: 6,tf-idf: 7,tf-idf: 8,tf-idf: 9,...,tf-idf: 3297,tf-idf: 3298,tf-idf: 3299,tf-idf: 3300,tf-idf: 3301,tf-idf: 3302,tf-idf: 3303,tf-idf: 3304,Relevancy,Urgency
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.323127,0.0,0.0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,3,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0,0


In [153]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import * 
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import *
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

def get_stats(model, X, y, cv, verbose=False):
    accuracy = []
    precision = []
    recall = []
    f1 = []
    auc = []
        
    cv_results = cross_validate(model, X, y, scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc'], 
                                cv=cv, return_train_score=False)
    
    new_results = {}
    for k,v in cv_results.items():
        new_results[k] = np.mean(v)
    
    if verbose:
        print(new_results)
    
    #now return the data
    return new_results

df3 = df2.copy()
df3.pop('Relevancy')
labels = df3.pop('Urgency').map(lambda x: 1 if x == 2 else 0)

model = AdaBoostClassifier()
get_stats(model, df3, labels, cv=5)

{'fit_time': 5.779833889007568,
 'score_time': 0.6838139057159424,
 'test_accuracy': 0.965751323323544,
 'test_precision': 0.33333333333333337,
 'test_recall': 0.15333333333333332,
 'test_f1': 0.1904812834224599,
 'test_roc_auc': 0.6524034292366184}